In [21]:
from minio import Minio
import json
import pandas as pd
from io import BytesIO
import re

minio_client = Minio(
    "minio:9000",
    access_key="minio",
    secret_key="SU2orange!",
    secure=False
)


In [4]:
def read_bucket_data(bucket_name):
    objects = minio_client.list_objects(bucket_name, recursive=True)
    data = []
    for obj in objects:
        response = minio_client.get_object(bucket_name, obj.object_name)
        content = json.load(BytesIO(response.read()))
        data.append(content)
    return data

def read_all_buckets():
    buckets = minio_client.list_buckets()
    all_data = []
    for bucket in buckets:
        bucket_name = bucket.name
        all_data.extend(read_bucket_data(bucket_name))
    return all_data

In [5]:
all_data = read_all_buckets()

In [16]:
df = pd.DataFrame(all_data)
df.dropna(subset=['title', 'content'], inplace=True)
df['publishedAt'] = pd.to_datetime(df['publishedAt'])

In [18]:
df['text'] = df['title'] + " " + df['content']
df = df[['text']]

In [19]:
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = text.strip()  # Remove leading/trailing spaces
    return text

In [22]:
df['text'] = df['text'].apply(preprocess_text)